#### 라이브러리

In [114]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import time
from sklearn.metrics import accuracy_score
import warnings
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [115]:
# 경고 메시지 무시
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

In [116]:
data = pd.read_csv('모델링DB용.csv')
data.head(5)

,curColor,curPer,curColor2,curPer2,curColor3,curPer3,curObject,curHead,curTitle,curExplain,curArtist,song,genre
0,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Like I Do,NaN,박재범,박재범 - Like I Do,R&B
1,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Rewind,NaN,VVON,VVON - Rewind,R&B
2,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Westcoast Marriage,NaN,Austn,Austn - Westcoast Marriage,R&B
3,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Unlock,NaN,차다빈,차다빈 - Unlock,R&B
4,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Beautiful,NaN,basecamp,basecamp - Beautiful,R&B


In [117]:
# 필요한 열 제외
data = data.drop(columns=['curArtist', 'curTitle'])

# 결측치 처리
data = data.fillna('')

# 범주형 변수를 category 타입으로 변환
categorical_cols = ['curColor', 'curColor2', 'curColor3', 'curObject', 'curHead', 'curExplain', 'genre', 'song']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# 'song' 열을 Label Encoding
label_encoder = LabelEncoder()
data['song'] = label_encoder.fit_transform(data['song'])

# 데이터 분리
X = data.drop(columns=['song'])
y = data['song']

# 학습용 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 모양 확인
print(X_train.shape, y_train.shape)

(5244, 10) (5244,)


In [118]:
# DMatrix 생성
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [119]:
# 모델 파라미터 설정
params = {
    'objective': 'multi:softmax',
    'learning_rate': 0.05,
    'max_depth': 6,
    'num_class': len(label_encoder.classes_)
}

# 학습 시간 측정
start_time = time.time()
model = xgb.train(params, dtrain, num_boost_round=100)
end_time = time.time()

# 학습 시간
training_time = end_time - start_time
print(f'학습 시간: {training_time} 초')

학습 시간: 146.63910484313965 초


In [120]:
# 모델 저장
with open('xgboost_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [121]:
# 모델 평가
y_pred = model.predict(dtest)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.006102212051868803


In [122]:
# 예측된 레이블을 원래 값으로 변환
y_pred_original = label_encoder.inverse_transform(y_pred.astype(int))

# 예측 결과 출력
print("예측된 결과 (원래 값으로 변환):")
print(y_pred_original)

예측된 결과 (원래 값으로 변환):
['PATEKO - Sometimes' 'Gervs - Another Day' 'Valley - Tempo' ...
 'Nieman - Brutally Honest' 'SOLE - RIDE' '크루셜스타 - 너의 밤엔']


In [123]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['curExplain'].astype(str))

#### 사용자 입력을 받아 예측하는 함수

In [124]:
import numpy as np

# 사용자 입력을 받아 예측하는 함수
def recommend_song(model, label_encoder, tfidf_vectorizer, tfidf_matrix, input_keywords):
    # 입력 키워드를 TF-IDF 벡터화
    input_vector = tfidf_vectorizer.transform([input_keywords])
    
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(input_vector, tfidf_matrix)
    
    # 유사도가 가장 높은 인덱스 찾기
    most_similar_idx = np.argmax(cosine_sim)
    
    # 가장 유사한 노래 추천
    recommended_song_idx = y[most_similar_idx]
    recommended_song = label_encoder.inverse_transform([recommended_song_idx])
    
    return recommended_song[0]

In [125]:
# 저장된 모델 불러오기
with open('xgboost_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [126]:
# 사용자 입력 예제
input_keywords = '신나는 운동중 바다'

# 예측 수행
recommended_song = recommend_song(loaded_model, label_encoder, tfidf_vectorizer, tfidf_matrix, input_keywords)
print(f"Recommended song: {recommended_song}")

Recommended song: HONNE - Woman
